In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News Site

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [4]:
browser.visit(url)

In [5]:
#Create BeautifulSoup object, parse with html parser
soup = bs(browser.html, 'html.parser')

In [6]:
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', 'article_teaser_body').text
print(news_title)
print(news_p)

NASA's Mars 2020 Heads Into the Test Chamber
In this time-lapse video taken at JPL, engineers move the Mars 2020 rover into a large vacuum chamber for testing in Mars-like environmental conditions.


# JPL Mars Space Images - Featured Image 

In [7]:
url = 'https://jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
soup=bs(browser.html, 'html.parser')

In [8]:
#scrape site for image
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23539-640x350.jpg


# Mars Weather

In [9]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
soup = bs(browser.html, 'html.parser')

In [10]:
mars_weather = soup.find('div', class_='js-tweet-text-container').text
print(mars_weather)


InSight sol 338 (2019-11-08) low -102.5ºC (-152.5ºF) high -24.0ºC (-11.1ºF)
winds from the SSE at 5.3 m/s (11.9 mph) gusting to 19.6 m/s (43.9 mph)
pressure at 6.90 hPapic.twitter.com/VxWNunPM5q



# Mars Facts

In [11]:
# Visit the Mars facts webpage and scrape table data into Pandas
url = "http://space-facts.com/mars/"
browser.visit(url)
soup = bs(browser.html, 'html.parser')

In [12]:
mars_info = pd.read_html(url)
mars_info = pd.DataFrame(mars_info[0])
#mars_info.columns = ['Attribute', 'Value']
#mars_info.set_index('Attribute', inplace=True)
#mars_info.head()

In [13]:
#convert to HTML table
mars_facts = mars_info.to_html(header = False, index = False)
mars_facts.replace('\n', '')
mars_facts


'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [14]:
#save table to file
#mars_facts.to_html('Mars_table.html')

# Mars Hemispheres

In [15]:
# Visit the USGS Astogeology site and scrape pictures of the hemispheres
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
soup=bs(browser.html, 'html.parser')


In [16]:

#Create empty dictionary
hemisphere_image_urls  = []

#Point to hemisphere product block
products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

#loop through each of the hemispheres
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup=bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    print(image_url)
    hemisphere_image_urls.append({"title": title, "image_url": image_url})
    
hemisphere_image_urls

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


[{'title': 'Cerberus Hemisphere ',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [17]:
mars_data = {
    "news_title": news_title,
    "news_p": news_p,
    "featured_image_url": featured_image_url,
    "mars_weather": mars_weather,
    "mars_facts": mars_facts,
    "featured_image_url": featured_image_url,
    "hemisphere_image_urls":hemisphere_image_urls
}
mars_data

{'news_title': "NASA's Mars 2020 Heads Into the Test Chamber",
 'news_p': 'In this time-lapse video taken at JPL, engineers move the Mars 2020 rover into a large vacuum chamber for testing in Mars-like environmental conditions.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23539-640x350.jpg',
 'mars_weather': '\nInSight sol 338 (2019-11-08) low -102.5ºC (-152.5ºF) high -24.0ºC (-11.1ºF)\nwinds from the SSE at 5.3 m/s (11.9 mph) gusting to 19.6 m/s (43.9 mph)\npressure at 6.90 hPapic.twitter.com/VxWNunPM5q\n',
 'mars_facts': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <

In [18]:
browser.quit()